In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import ndex2
import ndex2.client
from ndex2 import cx2
import cdapsutil
import numpy as np

In [ ]:
network_df = pd.read_table("/network_with_weights.edgelist",
                           header=None)
network_df

In [ ]:
network_df.columns=["source","target","weight"]
#network_df

In [ ]:
import networkx as nx

# Create a graph 

nxg = nx.Graph()

# Add edges from DataFrame
nxg.add_weighted_edges_from(network_df.itertuples(index=False, name=None))
nxg


In [ ]:
#conert nx to nicex
###  first convert to ndex

G_ndex = ndex2.create_nice_cx_from_networkx(nxg)

In [ ]:
# Create CommunityDetection object
#thi is the step that can take a bit - but not too bad
cd = cdapsutil.CommunityDetection()
G_hier = cd.run_community_detection(G_ndex, algorithm='hidefv1.1beta',arguments={'--maxres':'300'}) #play around with max res param or other params here


In [ ]:

#back to networkx
G_hier_net = G_hier.to_networkx(mode='default')


In [ ]:
nodes = G_hier_net.nodes()
len(nodes)

In [ ]:

#find which proteins are in which 'node' / community
hier_df = pd.DataFrame.from_dict(dict(G_hier_net.nodes(data=True)), orient='index')
hier_df['system_ID']=hier_df.index.tolist()
hier_df['CD_MemberList_Size']=[int(x) for x in hier_df['CD_MemberList_Size'].tolist()]
hier_df['HiDeF_persistence']=[int(x) for x in hier_df['HiDeF_persistence'].tolist()]
#hier_df.head()


In [ ]:
hier_df

In [ ]:
plt.hist(hier_df['CD_MemberList_Size'])

In [ ]:
trimmed=[]
for i in hier_df['CD_MemberList_Size']:
    if i<25:
        trimmed.append(i)

In [ ]:
plt.hist(trimmed)

In [ ]:
min(trimmed)

In [ ]:
trimmed

# first filter then trim

In [ ]:
onecluster = pd.read_csv("/onecluster.csv", index_col=0)
onecluster
keep_list = list(onecluster["xxx"])
len(keep_list)

In [ ]:
filtered_communiities = []
for st in hier_df['CD_MemberList']:

    i = st.split()
    new_i = []
    for nod in i:
        if nod in keep_list:
            new_i.append(nod)
    if len(new_i) > 0:
        filtered_communiities.append(new_i)

len(filtered_communiities)


In [ ]:
com_size = []
ones = 0
for community in filtered_communiities:
    com_size.append(len(community))
    if len(community) == 1:
        ones += 1

plt.hist(com_size)
plt.xlabel("community size")
plt.ylabel("Frequency")
plt.title("Communities size distribution before discarding large ones")

In [ ]:
label_list = []
nodes_list = []
label = 1
for community in filtered_communiities:
    for nod in community:
        label_list.append("community " + str(label))
        nodes_list.append(nod)
        #combined_G.nodes[nod]["community label"]="community "+str(label)
    label += 1
len(nodes_list)

In [ ]:



trimmed_filtered_communiities = []
for i in filtered_communiities:
    #print("comm: " ,i)
    if len(i) < 50:
        #print("len comm: :", len(i))
        trimmed_filtered_communiities.append(i)

com_size = []
ones = 0
for community in trimmed_filtered_communiities:
    com_size.append(len(community))
    if len(community) == 1:
        ones += 1
plt.hist(com_size)
plt.xlabel("community size")
plt.ylabel("Frequency")
plt.title("Communities size distribution after discarding large ones")

In [ ]:
min(com_size)

In [ ]:
label_list = []
nodes_list = []
label = 1
for community in trimmed_filtered_communiities:
    for nod in community:
        label_list.append("community " + str(label))
        nodes_list.append(nod)
        #combined_G.nodes[nod]["community label"]="community "+str(label)
    label += 1
len(nodes_list)

In [ ]:

output_30 = pd.DataFrame({"id": range(len(nodes_list)),
                             "xxx": nodes_list,
                             "prediction": label_list})


In [ ]:
output_30

In [ ]:

output_30.to_csv("/output 30.csv", index=False)

In [ ]:
#overlap_with_keep list
overlap_with_keep_list=[]
for prn in keep_list:
    if prn in nodes_list:
        overlap_with_keep_list.append(prn)

In [ ]:
len(overlap_with_keep_list)

In [ ]:
len(set(overlap_with_keep_list))

In [ ]:
445/881